# Importing epoch data from Fieldtrip to MNE

Importing data from fieldtrip to mne is not a trivial question. MNE has many powerful visualization, decoding and source localization tools that require your data to be in a specific data format (class). 

The major classes accepted in the MNE toolbox are listed in their website. [There is a good page on the Fieldtrip website explaining how to export data from Fieldtrip to MNE](http://www.fieldtriptoolbox.org/development/integrate_with_mne), however this is still in development: you can easily export data from fieldtrip to MNE if they are on the format of RAW or EVOKED, but not EPOCHS. Unfortunately for us, the epoched data type is specially interesting as it is the format the data is after being cleaned by fieldtrip.

For a long time in my lab rotation, it was really hard for me to import data from fieldtrip to Python in a way I could keep all the important features I wanted (the data, the time window, the fs, etc) and to be adapted to the MNE-workflow. After trying many codes that did not succeed, I finally found someone who had the same problem as mine. I could adapt their codes with what I had tried so far. This is what this tutorial is about: the function load_cor. Importantly, the code used in this tutorial is supposed to run on Windows. A small adaptation of function load_cor was created for mac users.

---

Let's start having a look on the function itself:

In [7]:
## This code is meant to work on WINDOWS. Giulia has a version that works on MAC. 

def load_cor(xDir, var_name='struct_cor'): # the inputs of the function  are the directory of your variable
                                           # and its name.
        
    # first we import some of the tools that will be used in this function.
    # Even if you import some of these tools in your main code, it is important
    # to keep these importations in the definition of the function so you don't
    # get an error message in case you forget to import a tool, for instance.
    import mne
    from mne import create_info
    from mne.epochs import EpochsArray
    import scipy.io as sio
    import numpy as np
    
    # load Matlab/Fieldtrip data
    mat = sio.loadmat(xDir, squeeze_me=True, struct_as_record=False) # here we use the loadmat to load the data
    ft_data = mat[var_name] # loadmat gives us a variable containing many lists, that include things we are not
                            # interested as the header of the file. We here retrieve only the part of the variable
                            # we are interested.
    # ft_data is now a python object that is inspired in the matlab structure you wanted to load. For instance, if in
    # matlab your structure had a field named "example", in python you will retrieve this by typing ft_data.example. 
    # Remember that, as objects, ft_data is not shown in the variable explorer of spyder.
    
    # Now we define the events. In this function, the events will be the different directions of the FIRST sound. 
    # This is retrieved in the trialinfo column 1. 
    events = ft_data.trialinfo[:, 1]


    # Now we can convert to mne.
    
    n_trial, n_chans, n_time = ft_data.trial.shape # we get the shape of the field "trial" of the structure ft_data
    
    # The next steps could have been done in one line by saying data = ft_data.trial
    # I used so many lines because I think I was still learning python and I wasn't always efficient.
    data = np.zeros((n_trial, n_chans, n_time))  # we create a empty variable
    data = ft_data.trial # and fill it with data.
    
    
    # we define the sampling frequency
    sfreq = 200
    # we get the time for the data
    time = ft_data.time

    # coi stands for channels of interest. In case we wanted to select specific channels (for instance, if we had
    # recorded EEG and MEG simultaneously), we could change coi to select which channels we are interested
    # by putting the index of channels we want. We are getting all the channels so this is not really 
    # important for us. But I left it here in case one day you decide to look at specific channels, you can easily
    # change it here
    coi = range(n_chans)
    data = data[:, coi, :]
    
    # Now we encode the channels names. This is important when defining the montage, for instance. In MAC, this
    # steps yields something like "b'E01" instead of "E01". I couldn't understand why (I've never used mac in my life)
    # but my guess is that it has to do with the ascii encoding. Anytheless this is easily solvable by removing the "b'"
    # from all channels (which we don't do in this example).
    
    chan_names = [l.encode('ascii') for l in ft_data.label[coi]]
    chan_types = ft_data.label[coi]
    chan_types[:] = 'eeg'
    # we then create the info of our MNE epoch classe.
    info = create_info(chan_names, sfreq, chan_types)
    # We also arrange the events. The events variable has to be matrix of 3 cols. The first column is the
    # trial ID, the second col is the previous event in that trial, and the third columns is the actual event that is
    # happening. This means that we can have more than one event per trial. I wanted to keep it simple, so I defined
    # my events to be only the position of the first sound, but this could be definitely be improved to
    # add more labels if you want. To understand how the events in MNE work better, I recommend the MNE website
    events = np.array([np.arange(n_trial), np.zeros(n_trial), event, ], int).T
    
    # We then finally use the MNE function EpochsArray to create and EPOCH class.
    epochs = EpochsArray(data, info, events=events,
                         tmin=np.min(time), verbose=False)
    
    # we also put a montage of our EEG system so we can plot the topographies.
    montage = mne.channels.read_montage('GSN-HydroCel-257')
    epochs.set_montage(montage)
    return epochs, event # those are the outputs of the functions are the class of epochs and the event matrix.
    # REMEBER: epochs won't show in the variable explorer of the spyder.

Now that we understand the function inside, let's try an example and see if it works:

In [8]:
xDir = 'C:\\Users\\Ana\\Desktop\\CI\\Python\\Subjects\\AB171109_20171109_111227v03HT\\struct_cor.mat' ## Example data
    
epochs, event = load_cor(xDir, var_name='struct_cor') # using the function

Now let's have a look how the class epochs looks like: type epochs and observe the output:

In [9]:
epochs

<EpochsArray  |  n_events : 750 (all good), tmin : -0.3 (s), tmax : 2.75 (s), baseline : None, ~759.2 MB, data loaded,
 '2': 150, '3': 150, '4': 150, '5': 150, '6': 150>

So epochs has n_events that should be equal to the number of trials for that subject. The events in the second line show how many trials were part of each direction. So direction '2' has x trials, direction 3 has y trials and so on. 

You can do many things with an epoch class. If you want to retrieve the EEG data and have a look at the info, for instance, you can use:

In [12]:
data = epochs.get_data()
info = epochs.info

print(info)

<Info | 17 non-empty fields
    bads : list | 0 items
    ch_names : list | E1, E2, E3, E4, E11, E12, E13, E14, E15, ...
    chs : list | 217 items (EEG: 217)
    comps : list | 0 items
    custom_ref_applied : bool | False
    dev_head_t : Transform | 3 items
    dig : list | 220 items
    events : list | 0 items
    highpass : float | 0.0 Hz
    hpi_meas : list | 0 items
    hpi_results : list | 0 items
    lowpass : float | 100.0 Hz
    meas_date : ndarray | 1970-01-01 00:00:00 GMT
    nchan : int | 217
    proc_history : list | 0 items
    projs : list | 0 items
    sfreq : float | 200.0 Hz
    acq_pars : NoneType
    acq_stim : NoneType
    buffer_size_sec : NoneType
    ctf_head_t : NoneType
    description : NoneType
    dev_ctf_t : NoneType
    experimenter : NoneType
    file_id : NoneType
    hpi_subsystem : NoneType
    kit_system_id : NoneType
    line_freq : NoneType
    meas_id : NoneType
    proj_id : NoneType
    proj_name : NoneType
    subject_info : NoneType
    xplo

If you would like, on other hand, to plot the data to visualize it better, you can try: 

In [ ]:
epochs.plot_image()

750 matching events found
0 projection items activated
0 bad epochs dropped


A comprehensive list of what can be done witht the epochs class can be found [here](https://martinos.org/mne/stable/generated/mne.Epochs.html)

In [ ]:
epochs.plot()